In [ ]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)
tf.test.is_gpu_available()
gpus = tf.config.list_physical_devices('GPU')
if gpus: 
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=14336 )] #7168  6144
    )

logical_gpus = tf.config.list_logical_devices('GPU')
print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")

In [ ]:
#Kütüphanelerin eklenmesi
import locale
locale.getpreferredencoding = lambda: "UTF-8"
import numpy as np #Bu kütüphane lineer cebir için kullandığımız kütüphane fonksiyonlarını içeriyor
import pandas as pd # verilerimizi işlemek için pandas kütüphanasini kullanıyoruz(örn pd.read_scv)
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression
from sklearn import svm

from sklearn.model_selection import KFold, cross_val_predict, cross_val_score
!pip install utils
from utils import *
import random
import warnings
warnings.filterwarnings('ignore') 
#from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score, make_scorer,precision_score,recall_score,f1_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
import nltk
nltk.download('punkt')
from spacy.lang.tr import Turkish
from tqdm import tqdm
import string
from keras.preprocessing.text import Tokenizer
import tensorflow as tf
from keras.utils import np_utils
!pip install transformers
!pip install bert-tensorflow
from transformers import AutoModel, AutoTokenizer
import nltk 
import torch
import tqdm
import math
import bert
from tensorflow.keras import layers
from transformers import AutoModel, AutoTokenizer
import re
import random

In [ ]:
column = ['tweets','duygu']
df = pd.read_excel("../dataset/kemik_pos_neg.xlsx")
df.columns=column
#veri setinin gösterilmesi
df=df.drop_duplicates()
df['tweets']=df['tweets'].str.lower()


In [ ]:
df=df.sample(frac=1).reset_index(drop=True)

In [ ]:
def classification_report_with_accuracy_score(y_true, y_pred):
    print(classification_report(y_true, y_pred)) # print classification report
    
    return accuracy_score(y_true, y_pred) # return accuracy score

In [ ]:
def classification_report_with_precision_score(y_true, y_pred):
    
    return precision_score(y_true, y_pred, average='weighted')

In [ ]:
def classification_report_with_recall_score(y_true, y_pred):
    
    return recall_score(y_true, y_pred,average='weighted')

In [ ]:
def classification_report_with_f1_score(y_true, y_pred):
    
    return f1_score(y_true, y_pred,average='weighted')

In [ ]:
def classification_report_with_mcc_score(y_true, y_pred):
    
    return matthews_corrcoef(y_true, y_pred)

In [ ]:
def classification_report_with_c_kappa(y_true, y_pred):

    return cohen_kappa_score(y_true, y_pred)

In [ ]:
model = AutoModel.from_pretrained("dbmdz/bert-base-turkish-cased").to('cpu')
 
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased",do_lower_case=True)

In [ ]:
def batched_embeddings(text_list, batch_size = 128):
    count=0
    pooled_outputs = []
    encoding = tokenizer(text_list, max_length=256, truncation=True, padding="max_length")

    for i in range(len(encoding['input_ids'])//batch_size + 1):
        count+=1
        up_to = i*batch_size + batch_size

        if len(encoding['input_ids']) < up_to:
            up_to = len(encoding['input_ids'])

        input_ids = torch.LongTensor(encoding['input_ids'][i*batch_size:up_to])
        attention_mask = torch.LongTensor(encoding['attention_mask'][i*batch_size:up_to])
        with torch.no_grad(): 
            length  = torch.tensor(len(list(filter(lambda token_id: token_id != 0, input_ids.tolist()[0]))))
            outputs = model(
                input_ids=input_ids[:, :length].to('cpu'), 
                attention_mask=attention_mask[:, :length].to('cpu'),)
            hidden_states = outputs[0].sum(dim=1)
            sentence_embedding = torch.div(hidden_states.permute(1, 0), length)
            pooled_outputs.extend(sentence_embedding.permute(1, 0).cpu())
            print(count)
    return torch.stack(pooled_outputs)

In [ ]:
y_total= []
y_total=df.duygu.tolist()

X_total = []
bert_emb=batched_embeddings(df.tweets.tolist(), batch_size = 128)
X_total= list([np.array(t,dtype=float) for t in bert_emb])

In [ ]:
# Nested CV with parameter optimization
svc = svm.SVC(kernel='linear')
clf = OneVsRestClassifier(svc)

accuracy = cross_val_score(clf, X_total, y_total, cv=3, \
               scoring=make_scorer(classification_report_with_accuracy_score))
print("Accurancy for bert/SVM: ",accuracy.mean())


precision = cross_val_score(clf, X_total, y_total, cv=3, \
               scoring=make_scorer(classification_report_with_precision_score))
print("Precision for bert/SVM: ",precision.mean())


recall = cross_val_score(clf, X_total, y_total, cv=3, \
              scoring=make_scorer(classification_report_with_recall_score))
print("Recall for bert/SVM: ",recall.mean())


f1 = cross_val_score(clf, X_total, y_total, cv=3, \
               scoring=make_scorer(classification_report_with_f1_score))
print("F1-Score for bert/SVM: ",f1.mean())

mcc_score = cross_val_score(clf, X_total, y_total, cv=3, \
               scoring=make_scorer(classification_report_with_mcc_score))

print("MCC_score for bert/SVM: ",mcc_score.mean())

c_kappa = cross_val_score(clf, X_total, y_total, cv=3, \
               scoring=make_scorer(classification_report_with_c_kappa))

print("C_kappa for bert/SVM: ",c_kappa.mean())

In [ ]:
clf =  LogisticRegression(multi_class='ovr')

accuracy = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_accuracy_score))
print("Accurancy for word2vec/LogisticRegression: ",accuracy.mean())


precision = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_precision_score))
print("Precision for word2vec/LogisticRegression: ",precision.mean())


recall = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_recall_score))
print("Recall for word2vec/LogisticRegression: ",recall.mean())


f1 = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_f1_score))
print("F1-Score for word2vec/LogisticRegression: ",f1.mean())

mcc_score = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_mcc_score))

print("MCC_score for word2vec/LogisticRegression: ",mcc_score.mean())

c_kappa = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_c_kappa))

print("C_kappa for word2vec/LogisticRegression: ",c_kappa.mean())

In [ ]:
"""clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(10, 3), random_state=1)

accuracy = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_accuracy_score))
print("Accurancy for bert/MLPC: ",accuracy.mean())


precision = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_precision_score))
print("Precision for bert/MLPC: ",precision.mean())


recall = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_recall_score))
print("Recall for bert/MLPC: ",recall.mean())


f1 = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_f1_score))
print("F1-Score for bert/MLPC: ",f1.mean())
"""

In [ ]:
clf = GaussianNB()
from sklearn.naive_bayes import BernoulliNB
clf=BernoulliNB()

accuracy = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_accuracy_score))
print("Accurancy for word2vec/NB: ",accuracy.mean())


precision = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_precision_score))
print("Precision for word2vec/NB: ",precision.mean())


recall = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_recall_score))
print("Recall for word2vec/NB: ",recall.mean())


f1 = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_f1_score))
print("F1-Score for word2vec/NB: ",f1.mean())

mcc_score = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_mcc_score))

print("MCC_score for word2vec/NB: ",mcc_score.mean())

c_kappa = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_c_kappa))

print("C_kappa for word2vec/NB: ",c_kappa.mean())